# The Price of Art

## Question
What determines the price of (modern) art?

## Hypothesis
An interaction of artist, artwork features, and representation.

## Methodology

1. Examine a model where price is just being predicted by artist, to see if that model can account for significant amounts of variance. 
2. Examine a model where price is just being predicted by artwork features, """ 
3. Examine a model where price is just being predicted by representation, """
4. See if we can do some structural equation modelling of this

### First we need to import and clean the data.

In [18]:
import numpy as np
import pandas as pd

# import from raw CSV
artsy_data = pd.read_csv("./data/artworks_list_FINAL.csv", header=0)
artsy_data = artsy_data.drop(columns=['collection_index'])

f"Our raw dataset has {len(artsy_data)} entries."

'Our raw dataset has 2970 entries.'

### Then we can remove any error rows.
These are entries where we were unable to collect useable data, most likely because the artwork had already been sold at the time of the data scraping, even though it was in the Artsy listings index.

In [19]:
# find rows where error collection has failed
error_rows = artsy_data.index[(artsy_data == "error").any(axis=1)].tolist()

# exclude these rows from the overall dataframe
artsy_listings = artsy_data.drop(error_rows, axis=0)

# how much of original dataframe did we exclude?
rows_excluded = len(error_rows)
percent_excluded = round((len(artsy_data) - len(artsy_listings)) / len(artsy_data) * 100, 2)
f"We excluded {rows_excluded} entries, or {percent_excluded}% of the original dataset, because of errors in data collection."

'We excluded 9 entries, or 0.3% of the original dataset, because of errors in data collection.'

In [20]:
# get some summary statistics
artsy_listings.describe()

,page_url,artist,artist_nationality,artist_birthdate,title,image_url,year,gallery,gallery_location,medium,medium_details,size_inches,size_cm,condition,classification,signed,authenticated,framed,currency,price
count,2961,2961,2929,2002,2961,2961,2957,2960,2960,2961,2961,2961,2961,1341,2961,2821,2090,2961,2736,2736
unique,2953,1150,339,77,2727,2953,364,411,437,16,1330,1799,1862,323,235,782,1,2,6,509
top,https://www.artsy.net/artwork/max-weber-reclin...,Peter Max,American,1963,Untitled,https://d32dm0rphc51dk.cloudfront.net/rKcV4uqy...,2020,Make offer,Certificate of authenticity,Painting,Oil on canvas,This work is part of a limited edition set.,50.8 × 40.6 cm,Excellent,This is a unique work.,Hand-signed by artist,Included,Not included,$,his work is part of a limited edition set.
freq,2,68,1046,113,117,2,744,749,748,1675,211,105,32,275,2477,1155,2090,1882,1794,100


### Examine data types, transform any that aren't right, and make appropriate data exclusions.

In [21]:
# look at data types
artsy_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2961 entries, 0 to 2969
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   page_url            2961 non-null   object
 1   artist              2961 non-null   object
 2   artist_nationality  2929 non-null   object
 3   artist_birthdate    2002 non-null   object
 4   title               2961 non-null   object
 5   image_url           2961 non-null   object
 6   year                2957 non-null   object
 7   gallery             2960 non-null   object
 8   gallery_location    2960 non-null   object
 9   medium              2961 non-null   object
 10  medium_details      2961 non-null   object
 11  size_inches         2961 non-null   object
 12  size_cm             2961 non-null   object
 13  condition           1341 non-null   object
 14  classification      2961 non-null   object
 15  signed              2821 non-null   object
 16  authenticated       2090

### Let's start with our target variable, price.

We will standardise price in GBP.

In [22]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['price'])

0                                           55,000
1                                            3,500
2                                            2,000
3                                            6,500
5                                            1,250
6                                           15,000
7                                            1,800
8                                              700
9                                            8,000
10                                          11,000
11                                           2,000
12                                           1,000
13                                           3,650
14                                           4,500
15                                           3,200
17                                           2,200
18                                             800
19                                          80,000
20                                           4,500
21                             

In [23]:
# remove all commas from the prices
artsy_listings['price'] = artsy_listings['price'].replace(',','', regex=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['price'])

0                                            55000
1                                             3500
2                                             2000
3                                             6500
5                                             1250
6                                            15000
7                                             1800
8                                              700
9                                             8000
10                                           11000
11                                            2000
12                                            1000
13                                            3650
14                                            4500
15                                            3200
17                                            2200
18                                             800
19                                           80000
20                                            4500
21                             

In [24]:
# convert all strings into numerics where possible, otherwise fill with NaNs
def convertNumeric(x):
    if type(x) == str:
        if x.isnumeric():
            x = float(x)
        else:
            x = np.NaN
    return x  

# change the price list to the updated values
artsy_listings['price'] = artsy_listings['price'].apply(lambda x: convertNumeric(x))

# take a look and do a summary
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['price'])

artsy_listings['price'].describe()

0        55000.0
1         3500.0
2         2000.0
3         6500.0
5         1250.0
6        15000.0
7         1800.0
8          700.0
9         8000.0
10       11000.0
11        2000.0
12        1000.0
13        3650.0
14        4500.0
15        3200.0
17        2200.0
18         800.0
19       80000.0
20        4500.0
21        3495.0
22         800.0
23         800.0
24         695.0
25        3500.0
26        1800.0
27        3900.0
28      120000.0
29        2000.0
30       16000.0
31         250.0
32        1900.0
33        1000.0
34        2400.0
35       18000.0
36        6500.0
37       23000.0
38         390.0
39       22000.0
40         250.0
41       13000.0
42        1500.0
43       50000.0
44       12900.0
45        1725.0
46        3200.0
47        1650.0
48        6000.0
49        5500.0
50       22500.0
51       45000.0
52       20000.0
53         200.0
54       10320.0
55       31250.0
56         580.0
57        1000.0
58         800.0
59       17000.0
60        1300

count      2513.000000
mean      14264.425388
std       45704.491797
min          32.000000
25%        2000.000000
50%        4750.000000
75%       12500.000000
max      850000.000000
Name: price, dtype: float64

Now we take a look at currency so we can standardise all these prices into GBP at today's market rate.

In [25]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings[['currency', 'price', 'gallery', 'gallery_location']])

     currency     price                                            gallery  \
0           $   55000.0                              Corridor Contemporary   
1           €    3500.0                                         Make offer   
2           €    2000.0                                          Artistics   
3           $    6500.0                                       Gallery 1261   
5           $    1250.0                                    Signari Gallery   
6           $   15000.0                                        Los Angeles   
7           $    1800.0                                       Gallery 1261   
8           $     700.0                                     Bakker Gallery   
9           €    8000.0                                         Make offer   
10          $   11000.0                          Luis De Jesus Los Angeles   
11          $    2000.0                                           New York   
12          $    1000.0                                       Ga

So we've got a few cases here:

1. Most artwork prices are in USD ($), GBP (£), or EUR (€).
2. We've got two other characters occasionally - "T" and "E" - which mark instances where the information about price was either not available or not in the standard price section. We will exclude these entries.
3. NaN entries - to be excluded.

In [37]:
# What are the rates for USD > GBP and EUR > GBP?
from currency_converter import CurrencyConverter
from datetime import date
c = CurrencyConverter()

# choose a date around when the data was collected
# default is the most recent rate available, but more recent dates in May 2021 are not available yet
c.convert(100, 'EUR', 'GBP', date=date(2021,3,1))
c.convert(100, 'USD', 'GBP', date=date(2021,3,1))

def currencySignConvert(sign, price):
    if sign == "$":
        price_GBP = c.convert(price, 'USD', 'GBP')
    elif sign == "€":
        price_GBP = c.convert(price, 'EUR', 'GBP')
    elif sign == '£':
        price_GBP = price
    else:
        price_GBP = np.NaN

    return price_GBP


## apply function to every row of the dataframe
price_GBP = []
for index, row in artsy_listings.iterrows():
    # print(row['currency'])
    # print(row['price'])
    price_GBP.append(round(currencySignConvert(row['currency'], row['price']),2))
artsy_listings['price_GBP'] = price_GBP

# describe the data
artsy_listings['price_GBP'].describe()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings[['currency', 'price', 'price_GBP']])

count      2513.000000
mean      11058.887577
std       36315.699465
min          22.720000
25%        1490.990000
50%        3549.990000
75%        9621.590000
max      689720.000000
Name: price_GBP, dtype: float64

### Next up are the predictor variables. 

Let's start with the artwork side, specifically, **year** (and therefore age) of the artwork.

In [80]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['year'])

0                                                ca. 2012
1                                                    2017
2                                                    2015
3                                                    2016
5                                                    2019
6                                                    2020
7                                                    2020
8                                            Late 20th c.
9                                                    2017
10                                                     II
11                                                   2018
12                                                   2020
13                                                   2020
14                                                   2020
15                                                   2017
17                                                   2020
18                                                   2020
19            

In [ ]:
# # remove 'ca.' or 'circa'

# # find entries containing 1 or 2, otherwise insert NaN

# # if entry contains "th" or "st":
#     early 19th --> 1825
#     (mid) 19th --> 1850
#     late 19th --> 1875

#     early 20th --> 1925
#     (mid) 20th --> 1950
#     late 20th --> 1975

#     21st --> 2010?
#     "Contemporary" --> 2010?

# # if two years separated by "/" or "-" --> take the later year (caution: some of the later years may only be double digits)

# # if year ends in "s" --> take year + 5


# convert year into an integer
# artsy_listings['year'].str.replace("ca. ", "").astype(int)
